In [1]:
include("../src/UNSflow.jl")

/Users/kiran/.julia/v0.5/Conda/deps/usr/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


UNSflow

In [2]:
using UNSflow

import UNSflow.mutual_ind

In [3]:
v1 = ThreeDVort([1;0;1;], [0.; 1.; 0.;], 0.02,[0.;0.;0.;], [0.;0.;0.;] )

UNSflow.ThreeDVort([1.0,0.0,1.0],[0.0,1.0,0.0],0.02,[0.0,0.0,0.0],[0.0,0.0,0.0])

In [4]:
v2= ThreeDVort([-1;1;0;], [0.; 1.; 0.;], 0.02,[0.;0.;0.;], [0.;0.;0.;] )

UNSflow.ThreeDVort([-1.0,1.0,0.0],[0.0,1.0,0.0],0.02,[0.0,0.0,0.0],[0.0,0.0,0.0])

In [55]:
function fgfromrhoGauss(rho::Float64)
    f = sqrt(2./pi)*exp(-rho^2/2.)
    g = erf(rho/sqrt(2.)) - rho*f
    return f, g
end

function fgfromrhoLeonard(rho::Float64)
    f = 7.5/(rho^2 + 1.)^3.5
    g = (rho^3*(rho^2 + 2.5))/(rho^2 + 1.)^2.5
    return f, g
end

function mutual_ind(vorts::Vector{ThreeDVort})
      for i = 1:length(vorts)
          for j = i+1:length(vorts)
            rvec = vorts[i].x - vorts[j].x
#              dx = vorts[i].x[1] - vorts[j].x[1]
#              dy = vorts[i].x[2] - vorts[j].x[2]
#              dz = vorts[i].x[3] - vorts[j].x[3]
              #source - tar

            r = norm(rvec)
              rhoi = r/vorts[i].vc
              rhoj = r/vorts[j].vc

            f, g = fgfromrhoLeonard(rhoi)
            
            vorts[j].v -= -g*cross(rvec,vorts[i].s)/(4.*pi*r^3)
                
              vorts[j].ds += (-g*cross(vorts[j].s, vorts[i].s)/rhoi^3 + 
            ((3*g/rhoi^3 - f)*dot(vorts[j].s, rvec)*cross(rvec,vorts[i].s))/r^2)/(4*pi*vorts[i].vc^3)  
                
            
            f, g = fgfromrhoLeonard(rhoj)
            vorts[i].v += -g*cross(rvec,vorts[j].s)/(4.*pi*r^3)
                
            vorts[i].ds += (-g*cross(vorts[i].s, vorts[j].s)/rhoj^3 + 
                ((3*g/rhoj^3 - f)*dot(vorts[i].s, rvec)*cross(rvec,vorts[j].s))/r^2)/(4*pi*vorts[j].vc^3)  
                
            

          end
      end
      return vorts
end

function mutual_ind_dble(vorts::Vector{ThreeDVort})
      for i = 1:length(vorts)
          for j = 1:length(vorts)
            if i != j     
                rvec = vorts[i].x - vorts[j].x
#              dx = vorts[i].x[1] - vorts[j].x[1]
#              dy = vorts[i].x[2] - vorts[j].x[2]
#              dz = vorts[i].x[3] - vorts[j].x[3]
              #source - tar

             r = norm(rvec)
         
              rhoj = r/vorts[j].vc

            f, g = fgfromrhoLeonard(rhoj)
            vorts[i].v += -g*cross(rvec,vorts[j].s)/(4.*pi*r^3)
                
            vorts[i].ds += (-g*cross(vorts[i].s, vorts[j].s)/rhoj^3 + 
                ((3*g/rhoj^3 - f)*dot(vorts[i].s, rvec)*cross(rvec,vorts[j].s))/r^2)/(4*pi*vorts[j].vc^3)  
                
            
            end
          end
      end
      return vorts
end

mutual_ind_dble (generic function with 1 method)

In [56]:
mutual_ind([v1; v2])

2-element Array{UNSflow.ThreeDVort,1}:
 UNSflow.ThreeDVort([1.0,0.0,1.0],[0.0,1.0,0.0],0.02,[0.00541456,0.0,-0.0108291],[0.00812184,0.0,-0.0162437])  
 UNSflow.ThreeDVort([-1.0,1.0,0.0],[0.0,1.0,0.0],0.02,[-0.00541456,0.0,0.0108291],[-0.00270728,0.0,0.00541456])

### Develop code to convect 3D vortices and visualise the results

In [57]:
nvort = 50

50

In [58]:
vorts = ThreeDVort[]
for i = 1:nvort
    x = [0.0; real(i)*0.05; 0.0]
    s = [0.0; 0.002; 0.0]
    vcore = 0.02
    v = [0.0; 0.0; 0.0]
    ds = [0.0; 0.0; 0.0]
    push!(vorts, ThreeDVort(x, s, vcore, v, ds))
end

     

dt = 0.015
ntime = 100
@time for i =1:ntime
    for j = 1:nvort
        vorts[j].v = zeros(3)
        vorts[j].ds = zeros(3)
    end
    mutual_ind(vorts)
    for j = 1:nvort
        vorts[j].x += vorts[j].v + dt
        vorts[j].s += vorts[j].ds + dt
    end
end

In [59]:
vorts[1].s

3-element Array{Float64,1}:
 0.0  
 0.002
 0.0  

In [60]:
#leapfrogging vortices

The important quantities that should be related are:
Vortex strength (m2/s)
vortex core (m)
time step (m/s)

strength/(core*dt) = parameter
my usual = 0.01/(0.02*0.015) = 33
shiwei's = 10/(1*0.1) = 100
shiwei's distance between vortices:
radius = 6
number of points = 24
circ = 2*pi*6 = 37.7
distance bet vort = 37.7/24 = 1.6



In [69]:
vorts = ThreeDVort[]
dt = 0.1
core = 1
ndiv = Int(48)
for i = 1:ndiv/2
    x = [0; 6*sind(15*i); 6*cosd(15*i)]
    s = 10*[0.0; -cosd(15*i); sind(15*i)]
    v = zeros(3)
    ds = zeros(3)
    push!(vorts, ThreeDVort(x, s, core, v, ds))
end
for i = Int(ndiv/2+1):ndiv
    x = [2; 6*sind(15*i); 6*cosd(15*i)]
    s = 10*[0.0; -cosd(15*i); sind(15*i)]
    v = zeros(3)
    ds = zeros(3)
    push!(vorts, ThreeDVort(x, s, core, v, ds))
end
    
dt = 0.1
ntime = 500
@time for i =1:ntime
    for j = 1:ndiv
        vorts[j].v = zeros(3)
        vorts[j].ds = zeros(3)
    end
    mutual_ind(vorts)
    for j = 1:ndiv
        vorts[j].x += vorts[j].v*dt
        vorts[j].s += vorts[j].ds*dt
    end
end

  2.080822 seconds (47.05 M allocations: 2.093 GB, 11.01% gc time)


In [70]:
vorts[1].s

3-element Array{Float64,1}:
   1.78747e-13
 -10.946      
   2.93297    

In [71]:
xs = zeros(ndiv)
ys = zeros(ndiv)
zs = zeros(ndiv)

for i = 1:ndiv
    xs[i] = vorts[i].x[1]
    ys[i] = vorts[i].x[2]
    zs[i] = vorts[i].x[3]
end

In [72]:
using PyPlot
ax = gca(projection="3d")
pygui(true)
scatter3D(xs, ys,zs,s=20,)
#s

PyObject <mpl_toolkits.mplot3d.art3d.Path3DCollection object at 0x32c494910>

Works when the computations are done both ways. The biot savart can be simply inverted in sign. The stretchig is probably more complex. Must be properly derived